# Classical Decision Theory Example

In this example, we'll solve the following problem that we saw in the lectures: should we visit the offshore turbine by boat in the next hour?

## Optimal Wald Strategies

We start with some type definitions (which you can ignore if you like).
We'll also make use of the ``expectation`` function that we saw earlier.

In [1]:
from collections.abc import Callable
from enum import Enum
from typing import Sequence

Param = Enum("Param", ["LOW", "HIGH"])  # avg height in next hour
Data = Enum("Data", ["LOW", "HIGH"])  # avg height from last hour
Decision = Enum("Decision", ["BOAT", "NO_BOAT"])  # whether to send a boat
Strategy = Callable[[Data], Decision]  # function from data to decision

def expectation(pmf: Sequence[float], gamble: Sequence[float]) -> float:
    return sum(p * g for p, g in zip(pmf, gamble))

We first specify the utility function $U(d, x)$: each combination of decision $d$ and parameter $x$ leads to a
different final reward value. We can only board the offshore turbine for maintenance if the parameter $x$ is low.
Taking boat costs €1000. Doing maintenance saves €4000. So, in units of €1000:

In [2]:
def utility(d: Decision, x: Param) -> float:
    match d, x:
        case Decision.BOAT, Param.LOW:
            return 3
        case Decision.BOAT, Param.HIGH:
            return -1
        case _:  # all other cases
            return 0

The likelihood encodes the probability of the data given the parameter, $p(y\mid x)$.
The prior encodes the probability of the parameter, $p(x)$.
They are given as follows:

In [3]:
def likelihood(y: Data, x: Param) -> float:
    # probability of data y given parameter x
    match y, x:
        case Data.LOW, Param.LOW:
            return 0.9
        case Data.HIGH, Param.LOW:
            return 0.1
        case Data.LOW, Param.HIGH:
            return 0.3
        case Data.HIGH, Param.HIGH:
            return 0.7

def prior(x: Param):
    match x:
        case Param.LOW:
            return 0.4
        case Param.HIGH:
            return 0.6

Wald's expected utility for a strategy $\delta$ given parameter value $x$ is:
$$U(\delta|x)=E(U(\delta(Y),x)|x)=\sum_y U(\delta(y),x)p(y|x)$$
We can code this as follows:

In [4]:
def wald_expected_utility(strategy: Strategy, x: Param) -> float:
    pmf = [likelihood(y, x) for y in Data]
    gamble = [utility(strategy(y), x) for y in Data]
    return expectation(pmf=pmf, gamble=gamble)

Let's now define four possible strategies for our problem:

* always take the boat,
* never take the boat,
* take the boat if the last hour's average wave height was low,
* take the boat if the last hour's average wave height was high.

These are encoded in by the functions below.

In [5]:
def strategy_boat(y: Data) -> Decision:
    return Decision.BOAT

def strategy_no_boat(y: Data) -> Decision:
    return Decision.NO_BOAT

def strategy_boat_if_low(y: Data) -> Decision:
    return Decision.BOAT if y == Data.LOW else Decision.NO_BOAT
    
def strategy_boat_if_high(y: Data) -> Decision:
    return Decision.BOAT if y == Data.HIGH else Decision.NO_BOAT

strategies: Sequence[Strategy] = [
    strategy_boat,
    strategy_no_boat,
    strategy_boat_if_low,
    strategy_boat_if_high,
]

Let's put everything together and calculate the Wald expected utilities for each strategy, as a function of the parameter $x$:

In [6]:
for strategy in strategies:
    print(strategy.__name__, [wald_expected_utility(strategy, x) for x in Param])

strategy_boat [3.0, -1.0]
strategy_no_boat [0.0, 0.0]
strategy_boat_if_low [2.7, -0.3]
strategy_boat_if_high [0.30000000000000004, -0.7]


Verify that ``strategy_boat_if_high`` is dominated by ``strategy_boat_if_low``. Which strategies are admissible?

## Optimal Bayes Strategies

In the Bayesian setting, we simply calculate the posterior expected utility. First, we need to find the posterior $p(x\mid y)$:
$$p(x|y)=\frac{p(y|x)p(x)}{\sum_{x'}p(y|x')p(x')}$$
In code, this is:

In [7]:
def posterior(x: Param, y: Data) -> float:
    return prior(x) * likelihood(y, x) / sum(prior(x_) * likelihood(y, x_) for x_ in Param)

Now we can calculate the posterior expected utility of any decision $d$ given the data $y$:
$$U(d|y)=E(U(d,X)|y)=\sum_{x}U(d,x)p(x|y)$$
In code:

In [8]:
def posterior_expected_utility(d: Decision, y: Data) -> float:
    pmf = [posterior(x, y) for x in Param]
    gamble = [utility(d, x) for x in Param]
    return expectation(pmf=pmf, gamble=gamble)

Let's put everything together.

In [9]:
for y in Data:
    for d in Decision:
        print(y, d, posterior_expected_utility(d, y))

Data.LOW Decision.BOAT 1.6666666666666667
Data.LOW Decision.NO_BOAT 0.0
Data.HIGH Decision.BOAT -0.6521739130434783
Data.HIGH Decision.NO_BOAT 0.0


Confirm that the optimal Bayes decision strategy is to take the boat when the average wave height in the last hour was low, and otherwise not to take the boat.

**Exercise** The prior used to find this strategy was $p(\text{low})=0.4$ and $p(\text{high})=0.6$. Repeat the calculation for $p(\text{low})=0$ and $p(\text{high})=1$, and again for $p(\text{low})=0$ and $p(\text{high})=1$. Thereby, confirm that the Bayesian analysis can recover all Wald admissible strategies for this problem.